# Домашнее задание 2. Сравнение систем автоматического распознавания речи.

В этом задании вам нужно будет сравнить автоматические субтитры YouTube с другими системами.
Можно использовать любой способ скачивания данных, который вам удобен.|


Дополнительные баллы выше 8 можно получить за использование библиотек ASR, 
которые не были показаны на семинаре, и другие возможные улучшения.

In [5]:
!pip install youtube-transcript-api -q
!pip install pytube -q
!pip install wget -q

!pip install huggingsound -q
!pip install SpeechRecognition -q
!pip install git+https://github.com/openai/whisper.git soundfile -q
!pip install vosk -q
!pip install transformers datasets evaluate jiwer -q

In [15]:
import json
import wget
import string
import evaluate
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube
from pydub import AudioSegment

## 1. Выберите любое видео на YouTube, где есть речь (на любом известном вам языке) и автоматические субтитры.

Я выбрала две библиотеки [pytube](https://pytube.io/en/latest/) и [youtube-transcript-api](https://pypi.org/project/youtube-transcript-api/). Первую для скачивания аудио, а вторую для скачивания субтитров. 
> *Простите, у меня не работает youtube-dl...*😭

In [7]:
# скачиваем аудио в формате mp4, потому что доступен только этот формат
url = 'https://youtu.be/CnCm1rSIE5U?si=qNoyGDDO5dOGBQV0'
video = YouTube(url)
stream = video.streams.filter(only_audio=True).first()
stream.download(filename='youtube.mp4')

'/kaggle/working/youtube.mp4'

In [8]:
# конвертируем аудио в формат wav
sound = AudioSegment.from_file('youtube.mp4', format='mp4')
sound.export('youtube.wav', format='wav')

<_io.BufferedRandom name='youtube.wav'>

In [9]:
# скачиваем автоматические субтитры
srt = YouTubeTranscriptApi.get_transcript('CnCm1rSIE5U', languages=['ru'])
autogenerated_subs = ' '.join([sr['text'] for sr in srt if not sr['text'].startswith('[')])
# записываем в файлы
with open('youtube_subs.json', 'w', encoding='utf-8') as json_file:
    json.dump(srt, json_file, ensure_ascii=False, indent=4)
with open('youtube_subs.txt', 'w', encoding='utf-8') as txt_file:
    txt_file.write(autogenerated_subs)

## 2. Выберите 3 системы ASR и примените их к вашему файлу со звуком. 
Используйте разные библиотеки - те, которые упоминались в семинаре, или любые другие. За использование 2 разных библиотек - 4 балла. Если вы прогоняете через 3 системы с использованием одной библиотеки (например, SpeechRecognition) - это 2 балла.

### Библиотека `SpeechRecognitionModel` от `huggingsound`
Я взяла [модель](https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-russian), которая, по сути, является фейсбучной Wav2Vec2-XLSR-53, отфайнтьюнленной на русский. 

In [10]:
from huggingsound import SpeechRecognitionModel

In [7]:
model_1 = SpeechRecognitionModel('jonatasgrosman/wav2vec2-large-xlsr-53-russian')

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [8]:
transcriptions_1 = model_1.transcribe(['youtube.wav'])
# записываем в файлы
with open('wav2vec2.json', 'w', encoding='utf-8') as json_file:
    json.dump(transcriptions_1[0], json_file, ensure_ascii=False, indent=4)
with open('wav2vec2.txt', 'w', encoding='utf-8') as txt_file:
    txt_file.write(transcriptions_1[0]['transcription'])

100%|██████████| 1/1 [05:51<00:00, 351.54s/it]


### Библиотека `SpeechRecognition` (Whisper от OpenAI)

In [9]:
import speech_recognition

In [11]:
recognition = speech_recognition.Recognizer()
with speech_recognition.AudioFile('youtube.wav') as source:
    audio = recognition.record(source)
    recognition.adjust_for_ambient_noise(source)

In [12]:
transcriptions_2 = recognition.recognize_whisper(audio, language="ru")
# записываем в файл
with open('whisper.txt', 'w', encoding='utf-8') as txt_file:
    txt_file.write(transcriptions_2)

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 94.5MiB/s]


### Библиотека `vosk`

In [13]:
!vosk-transcriber -l ru -i youtube.wav -o vosk.txt

vosk-model-small-ru-0.22.zip: 100%|████████| 44.1M/44.1M [00:04<00:00, 10.6MB/s]
LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /root/.cache/vosk/vosk-model-small-ru-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /root/.cache/vosk/vosk-model-small-ru-0.22/graph/HCLr.fst /root/.cache/vosk/vosk-model-small-ru-0.22/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /root/.cache/vosk/vosk-model-smal

### 3. Оцените качество работ ваших систем и автогенерации YouTube. 
Для этого вам нужен эталон. Вы можете отредактировать один из результатов ASR и использовать его в качестве эталона.
Затем посчитайте WER для каждой системы. - 2 балла

In [ ]:
wget.download('https://raw.githubusercontent.com/KatiaKozlova/nlp-hse/main/2023/АвтОбрЕЯ_Козлова_ДЗ2/golden_standart.txt')

In [35]:
wer = evaluate.load("wer")

In [32]:
def normalize(file_name):
    with open(file_name, 'r', encoding='utf-8') as my_file:
        transcript = my_file.read()
    transcript = ' '.join([item.lower().strip(string.punctuation) for item in transcript.split() if item.lower().strip(string.punctuation)])
    return [transcript]

In [37]:
golden = normalize('golden_standart.txt')
model_file = {'Autogenerated subs': 'youtube_subs.txt', 'Wav2Vec': 'wav2vec2.txt', 'Vosk': 'vosk.txt'}
for model in model_file:
    my_wer = wer.compute(predictions=normalize(model_file[model]), references=golden)
    print(f'{model}: {my_wer}')

Autogenerated subs: 0.23595505617977527
Wav2Vec: 0.5
Vosk: 0.24157303370786518
